# Adversarial Example

## Configuration

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
!pip install click

  Using cached click-8.0.1-py3-none-any.whl (97 kB)


In [6]:
import warnings

warnings.filterwarnings(action='ignore')

import torch

### local imports
import model


### Environment imports
import click # Class object() 대신 argument 조절 library 사용
import math
import os
import shutil # 고수준 파일 연산
import torch
import torch.utils.data.dataset
import torchvision

In [7]:
### Configuration settings
# Resnet의 기본 학습 단계이다. 
# Training rate / size parameters
TRAIN_BATCHSIZE = 128  # GPU 개수에 따라 배수로 조절해도 됨.
TRAIN_LR = 0.1  # GPU 개수에 따라 배수로 조절해도 됨. --> ?
TRAIN_MOMENTUM = 0.9 #  --> (SGD optimizer 사용)
TRAIN_WEIGHT_DECAY = 1e-4 # --> (SGD optimizer 사용)
TRAIN_EPOCHS = [170, 195, 200]  # Divide lr by 10 at each; finish after last. --> 해당 epoch이 될 때마다 학습률을 0.1배


In [8]:
# Adversarial training settings
# 적대적 훈련은 노이즈에 대한 L2 loss를 최소화함
# 동시에 번갈아가면서 class에서 멀어지도록 gradient ascent를 진행..
TRAIN_ADV_EPS = 0.01 # 적대적 설명을 사용한다.
TRAIN_ADV_L2MIN_EPS = 0.1 # 적대적 설명을 사용할 때 L2 MIN도 사용가능.

#### 추가적 이해 필요

In [9]:
# Adversarial robustness parameters
ROBUST_Z = 2
ROBUST_ZETA = 0.2  # 항상 쌍으로 이루어진다(tandem).
ROBUST_ADAPT_L_TARGET = 1.5
ROBUST_ADAPT_PSI_0 = 220
ROBUST_ADAPT_PSI = 0.02
ROBUST_ADAPT_EPS_POS = 1
ROBUST_ADAPT_EPS_NEG = 0.01

### How to reset?

In [10]:
# Offset as [mean, std] of data input. --> ( resnet 때문에 투입)
MODEL_INPUT_OFFSET = [[0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]]

### 네트워크 구조

In [14]:
def bdd100k_preprocess(ft_out):
    return torch.nn.Conv2d(3, ft_out, kernel_size=3, padding=1, bias=False)

In [16]:
MODEL_ARCH=[
    32, # input size, 정사각형 가정
    bdd100k_preprocess, #Resnet의 initial layer.
    [(44 - 2)//6 for _ in range(3)],# [7,7,7],  Block Length --> ( ... ) 
    [16, 32, 64], # feature의 개수들.
    11,  # class 개수 (BDD-1000k)
]

In [ ]:
# Low-resource computer를 위한 실험 (우선 배제)
# ONE_BATCH_ONLY = False


## dataset

In [11]:
class_list=['trailer', 'motorcycle', 'bicycle', 'car', 'bus', 'other vehicle', 'pedestrian', 'other person', 'truck', 'train', 'rider']

In [ ]:
root='../BDD100K_MOT2020_image/bdd100k/images/track/train_av'
root_test='../BDD100K_MOT2020_image/bdd100k/images/track/test_av'

In [ ]:
from torchvision.datasets import ImageFolder
train_dataset = ImageFolder(root='./dataset', transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=128, shuffle=True, num_workers=4)


In [ ]:
# # # # CIFAR-10 찾기
# # # # CIFAR10_PATH에 관한 환경변수가 없다면 None을 반환한다.
# # # cifar10_path = os.environ.get('CIFAR10_PATH', '')

# # if not cifar10_path.strip():
# # #     raise ValueError('Must specify CIFAR10_PATH environment variable.')

# cifar10_path='CIFAR10_PATH/'

# # 경로설정
# if not os.path.exists(cifar10_path):
#     os.mkdir(cifar10_path)
# # dataset 다운로드
# if len(os.listdir(cifar10_path))==0:
#     torchvision.datasets.CIFAR10(cifar10_path, download=True)

# # Labels
# class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog',
#         'frog', 'horse', 'ship', 'truck']

### main.py 호출